# Process the Addresses Data
1. Ingest the data into the data lakehouse - bronze_addresses
2. Perform data quality checks and transform the data as requied - silver_addresses_clean
3. Apply changes to the addresses data - silver_addresses

1. Ingest the data into the data lakehouse - bronze_addresses

In [0]:
-- This operation requires unity catalog

CREATE OR REFRESH STREAMING TABLE bronze_addresses
COMMENT 'Raw addresses data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS
SELECT
  *,
  _metadata.file_path AS input_file_path,
  CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/circuitbox/landing/operational_data/addresses',
  'csv',
  map('cloudFiles.inferColumntypes', 'true')
);

2. Perform data quality checks and transform the data as required - silver_addresses_clean

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_addresses_clean(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_address_line_1 EXPECT (address_line_1 IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_postcode EXPECT (LENGTH(postcode) = 5)
)
COMMENT 'cleaned addresses data'
TBLPROPERTIES('quality', 'true')
AS
SELECT
  customer_id,
  address_line_1,
  city,
  state,
  postcode,
  CAST(created_date AS DATE) AS created_date
FROM STREAM(LIVE.bronze_addresses)

3. Apply changes to the Addresses Data - silver_addresses

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_addresses
COMMENT 'SCD Type 2 addresses data'
TBLPROPERTIES("quality" = "true")

In [0]:
APPLY CHANGES INTO LIVE.silver_addresses
FROM STREAM(LIVE.silver_addresses_clean)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 2;